# Automating Input Generation

In the previous section, we generated the inputs for a calculation manually,
by reading from a previously written input file.

For more complex calculations though, it is likely that we would like to automate the input generation process.

In this section, we will see the some methods for generating the inputs for a `PwCalculation` automatically.

In [7]:
from local_module import load_temp_profile
from aiida import orm

data = load_temp_profile(add_sssp=True)
data

AiiDALoaded(profile=Profile<uuid='8674300fb80f416d95e3a0c9aa2af378' name='temp_profile'>, computer=<Computer: local_direct (localhost), pk: 1>, pw_code=<Code: Remote code 'pw.x' on local_direct, pk: 1, uuid: ad002c3d-22a1-4b9b-88c4-2e73dc66317b>, pseudos=SsspFamily<1>)

## Generating structures via ASE and Pymatgen

...

## Importing structures from external databases

In [5]:
from aiida.tools.dbimporters.plugins.cod import CodEntry
entry = CodEntry("http://www.crystallography.net/cod/1526655.cif")
structure = entry.get_aiida_structure()
structure.get_pymatgen()

Structure Summary
Lattice
    abc : 5.381 5.381 5.381
 angles : 90.0 90.0 90.0
 volume : 155.80772134100002
      A : 5.381 0.0 3.294912213105954e-16
      B : -3.294912213105954e-16 5.381 3.294912213105954e-16
      C : 0.0 0.0 5.381
PeriodicSite: Si (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: Si (1.3453, 1.3453, 1.3453) [0.2500, 0.2500, 0.2500]
PeriodicSite: Si (-0.0000, 2.6905, 2.6905) [0.0000, 0.5000, 0.5000]
PeriodicSite: Si (4.0358, 1.3453, 4.0357) [0.7500, 0.2500, 0.7500]
PeriodicSite: Si (2.6905, 0.0000, 2.6905) [0.5000, 0.0000, 0.5000]
PeriodicSite: Si (1.3452, 4.0358, 4.0357) [0.2500, 0.7500, 0.7500]
PeriodicSite: Si (2.6905, 2.6905, 0.0000) [0.5000, 0.5000, 0.0000]
PeriodicSite: Si (4.0358, 4.0358, 1.3453) [0.7500, 0.7500, 0.2500]

## Importing Pseudopotential families

```console
$ aiida-pseudo install sssp -x PBE -p efficiency -v 1.1
```

In [9]:
family = orm.Group.collection.get(label="SSSP/1.1/PBE/efficiency")
family

SsspFamily<1>

In [10]:
entry = CodEntry("http://www.crystallography.net/cod/1526655.cif")
structure = entry.get_aiida_structure()
family.get_pseudos(structure=structure)

{'Si': <UpfData: uuid: 4f40f659-186e-466d-b08a-cac54f4abd40 (pk: 28)>}

## Generating Kpoints by distance

In [17]:
import aiida_quantumespresso.calculations.functions.create_kpoints_from_distance as ks

entry = CodEntry("http://www.crystallography.net/cod/1526655.cif")
structure = entry.get_aiida_structure()
kpoints = ks.create_kpoints_from_distance(structure, orm.Float(0.2), orm.Bool(False))
kpoints.base.attributes.all

{'cell': [[5.381, 0.0, 3.294912213106e-16],
  [-3.294912213106e-16, 5.381, 3.294912213106e-16],
  [0.0, 0.0, 5.381]],
 'pbc1': True,
 'pbc2': True,
 'pbc3': True,
 'mesh': [6, 6, 6],
 'offset': [0.0, 0.0, 0.0]}